In [1]:
import os

import ipywidgets as widgets
from IPython.display import display

import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer, TFBertForSequenceClassification
from tqdm.notebook import tqdm
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import Recall
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from numba import cuda 
import mlflow
import mlflow.tensorflow
import ast

mlflow.set_tracking_uri("http://webengineering.ins.hs-anhalt.de:41004")
mlflow.set_experiment("SMART21: Resource Classifier")
mlflow.tensorflow.autolog()
    
strategy = tf.distribute.get_strategy()
device = cuda.get_current_device()

2021-10-04 22:28:42.282201: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
external_data = widgets.SelectMultiple(
    options=['DBpedia2020', 'LC-QuAD', 'DBpediaTranslated'],
    value=['DBpedia2020'],
    #rows=10,
    description='External Data:',
    disabled=False
)
display(external_data)

SelectMultiple(description='External Data:', index=(0,), options=('DBpedia2020', 'LC-QuAD', 'DBpediaTranslated…

In [3]:
model = widgets.RadioButtons(
    options=['bert-base-cased', 'bert-base-multilingual-cased'],
    description='Model:',
    disabled=False
)
display(model)

RadioButtons(description='Model:', options=('bert-base-cased', 'bert-base-multilingual-cased'), value='bert-ba…

In [4]:
def regular_encode(texts, tokenizer, maxlen=512):
    """
    encodes text for a model
    """
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [5]:
def build_model(transformer, max_len=512, hidden_dim=32, n_classes=1):
    """
    builds a model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    if n_classes == 2: # binary classification
        out = Dense(1, activation='sigmoid')(cls_token)
    else:
        out = Dense(n_classes, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    
    if n_classes > 2:
        model.compile(
            Adam(lr=1e-5),
            loss='categorical_crossentropy',
            metrics=['categorical_accuracy']
        )
    else:
        model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 10
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MODEL = model.value

In [7]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

## Prepare Data

In [8]:
def convert_str_to_resource_class(x):
    lst = ast.literal_eval(x)
    
    if len(lst) > 0:
        return lst[0]
    else:
        return "NULL"
    
def convert_df_for_resource_training(data_frame):
    data_frame = pd.DataFrame.copy(data_frame)
    data_frame = data_frame[data_frame.category == 'resource']
    data_frame.type = data_frame.type.apply(lambda x: convert_str_to_resource_class(x))
    data_frame = data_frame[data_frame.type != 'NULL']
    
    return data_frame

In [9]:
dbp_df = pd.read_csv("../../data/at/dbpedia/smarttask_dbpedia_train_cleaned.csv", sep='$')
lcquad_df = pd.read_csv("../../data/at/common/lcquad_train_types.csv", sep='$')
dbp_translated_df = pd.read_csv("../../data/at/dbpedia/task1_dbpedia_train_translated.csv", sep='$')

langs = ['de', 'es', 'zh', 'it', 'ro', 'vi', 'ru', 'fr', 'cs', 'jap']

dbp_tmp = dbp_translated_df.copy()

for l in langs:
    dbp_tmp['question'] = dbp_tmp[f'question_{l}']
    dbp_translated_df = dbp_df.append(dbp_tmp)
     
dbp_translated_df = dbp_translated_df[dbp_df.columns].sample(frac=1)

data_dict = {
    'DBpedia2020': dbp_df,
    'LC-QuAD': lcquad_df,
    'DBpediaTranslated': dbp_translated_df,
}

/home/ins-alex/.local/lib/python3.8/site-packages/pandas/core/frame.py:7134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(


In [10]:
def add_external_data(df):
    for k in external_data.value:
        df = df.append(data_dict[k][df.columns])
    
    return df

In [11]:
max_len = list()

for q in add_external_data(dbp_df).question.values:
    max_len.append(len(tokenizer.encode(q)))
    
MAX_LEN = np.array(max_len).max()
del max_len

In [12]:
MAX_LEN = 145

In [13]:
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
skf.get_n_splits(dbp_df.id, dbp_df.category)

3

In [14]:
train_list = []
test_list = []

for train_index, test_index in skf.split(dbp_df.id, dbp_df.category):
    # print("TRAIN:", train_index, "TEST:", test_index)
    train_df = add_external_data(dbp_df.iloc[train_index])
    test_df = dbp_df.iloc[test_index]
    train_df = train_df[~train_df.question.isin(test_df.question)]
    train_list.append(convert_df_for_resource_training(train_df))
    test_list.append(convert_df_for_resource_training(test_df))

In [15]:
train_list[0].shape

(13875, 4)

In [16]:
tf_train_list, tf_test_list, lens, y_test_list = list(), list(), list(), list()

encoder = LabelEncoder()
encoder.fit(train_list[0].type.values.tolist() + test_list[0].type.values.tolist() + train_list[1].type.values.tolist() + test_list[1].type.values.tolist())
np.save('../../data/bin/label_encoders/dbp_encoder_2020_lcquad_translated.npy', encoder.classes_)

for i in range(n_splits):
    x_train = regular_encode(train_list[i].question.values.tolist(), tokenizer, maxlen=MAX_LEN)
    x_test = regular_encode(test_list[i].question.values.tolist(), tokenizer, maxlen=MAX_LEN)

    y_train = train_list[i].type.values
    y_test = test_list[i].type.values

    # encode textual labels into corresponding numbers
    
    encoded_y_train = encoder.transform(y_train) 
    encoded_y_test = encoder.transform(y_test)
    dummy_y_train = np_utils.to_categorical(encoded_y_train, num_classes=encoder.classes_.shape[0])
    dummy_y_test = np_utils.to_categorical(encoded_y_test, num_classes=encoder.classes_.shape[0]) # one hot encoded
    
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_train, dummy_y_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_test, dummy_y_test))
        .batch(BATCH_SIZE)
    )
    
    tf_train_list.append(train_dataset)
    tf_test_list.append(test_dataset)
    lens.append(x_train.shape)
    y_test_list.append(encoded_y_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ins-alex/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2126: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2021-10-04 22:29:09.017484: I tensorflow/stream_executor/platform/default/dso_loader.cc:5

## Train Model

In [18]:
class Metrics(Callback):
    def __init__(self, val_data, average='micro', mlflow=None, **kwargs):
        super(Metrics, self).__init__(**kwargs)
        self.validation_data = val_data
        self.average = average
        self.mlflow = mlflow
        
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        batches = len(self.validation_data)
        total = batches * BATCH_SIZE

        y_pred = np.array([])
        y_true = np.array([])

        for xVal, yVal in self.validation_data:
            y_pred = np.append(y_pred, np.argmax(self.model.predict(xVal), axis=1)).astype('int32')
            y_true = np.append(y_true, np.argmax(yVal, axis=1)).astype('int32')
        
        _val_f1 = f1_score(y_true, y_pred, average=self.average)
        _val_recall = recall_score(y_true, y_pred, average=self.average)
        _val_precision = precision_score(y_true, y_pred, average=self.average)
        
        if self.mlflow:
            mlflow.log_metric("val_f1", _val_f1)
            mlflow.log_metric("val_precision", _val_recall)
            mlflow.log_metric("val_recall", _val_precision)
            
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        
        print(" — val_f1: %f — val_precision: %f — val_recall %f" % (_val_f1, _val_precision, _val_recall))
        return

In [19]:
os.environ["N_SPLIT"] = "0"
i = int(os.environ.get('N_SPLIT'))

with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN, n_classes=encoder.classes_.shape[0])

print(model.summary())

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=1,
    min_delta=0.001,
    restore_best_weights=True
)

with mlflow.start_run():
    mlflow.log_param("KG", "DBpedia")
    mlflow.log_param("EPOCHS", EPOCHS)
    mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
    mlflow.log_param("MAX_LEN", MAX_LEN)
    mlflow.log_param("MODEL", MODEL)
    mlflow.log_param("DATA", str(external_data.value))
    mlflow.log_param("n_split_idx", i)

    n_steps = lens[0][0] // BATCH_SIZE # determine number of steps per epoch
    
    train_history = model.fit(
        tf_train_list[i],
        steps_per_epoch=n_steps,
        validation_data=tf_test_list[i],
        callbacks=[early_stopping, Metrics(tf_test_list[i], mlflow=mlflow)],
        epochs=EPOCHS
    )

2021-10-04 20:56:35.104377: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-04 20:56:35.880857: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If yo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/home/ins-alex/.local/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 145)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 177853440 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 309)               237621    
Total params: 178,091,061
Trainable params: 178,091,061
Non-trainable params: 0
_________________________________________________________________
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possib

2021-10-04 20:56:42.155313: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-10-04 20:56:42.155333: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-10-04 20:56:42.156621: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-10-04 20:56:42.156811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-10-04 20:56:42.164717: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so


...

3150/3150 [==============================] - 1294s 407ms/step - loss: 2.7108 - categorical_accuracy: 0.4216 - val_loss: 1.8156 - val_categorical_accuracy: 0.6021
 — val_f1: 0.602076 — val_precision: 0.602076 — val_recall 0.602076
Epoch 2/10
3150/3150 [==============================] - 1336s 424ms/step - loss: 1.9044 - categorical_accuracy: 0.5577 - val_loss: 1.6299 - val_categorical_accuracy: 0.6442
 — val_f1: 0.644228 — val_precision: 0.644228 — val_recall 0.644228
Epoch 3/10
3150/3150 [==============================] - 1340s 426ms/step - loss: 1.5776 - categorical_accuracy: 0.6247 - val_loss: 1.7134 - val_categorical_accuracy: 0.6675
Restoring model weights from the end of the best epoch.
 — val_f1: 0.644228 — val_precision: 0.644228 — val_recall 0.644228
Epoch 00003: early stopping


2021-10-04 22:04:37.802388: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


/home/ins-alex/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /tmp/tmp8wloxsf6/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp8wloxsf6/model/data/model/assets
2021/10/04 22:05:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ins-alex/.local/lib/python3.8/site-packages/mlflow/tensorflow.py:805: UserWarning: Logging to MLflow failed: [Errno 13] Permission denied: '/data'"
2021/10/04 22:05:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ins-alex/.local/lib/python3.8/site-packages/mlflow/tensorflow.py:601: UserWarning: Logging to MLflow failed: [Errno 13] Permission denied: '/data'"


In [20]:
model.save(f'../../data/bin/resource_dbpedia_base_2020_lcquad_translated')

/home/ins-alex/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ../../data/bin/resource_dbpedia_base_2020_lcquad_translated/assets


INFO:tensorflow:Assets written to: ../../data/bin/resource_dbpedia_base_2020_lcquad_translated/assets
